In [1]:
# import numpy as np
# import scipy as sp
# %matplotlib inline
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [2]:
lines = []
with open("train.dat", "r") as fh:
    lines = fh.readlines()
#     classes = temp[0]
#     lines = temp[1]

In [3]:
classes = []
datalines = []
for s in lines:
    temp = s.split("\t")
    classes.append(temp[0])
    datalines.append(temp[1])
    
print(len(classes))
print(classes)
print(len(datalines))
# print(datalines[0])
# with open("test.dat", "r") as fh:
#     lines2 = fh.readlines()

14438
['4', '5', '2', '5', '4', '1', '5', '5', '3', '5', '5', '5', '2', '5', '5', '5', '1', '5', '1', '3', '1', '3', '1', '5', '3', '5', '5', '5', '3', '5', '3', '5', '1', '5', '4', '3', '1', '4', '3', '1', '1', '1', '5', '4', '2', '5', '5', '3', '1', '3', '1', '5', '5', '5', '5', '3', '5', '4', '3', '1', '3', '3', '5', '5', '3', '5', '1', '3', '4', '5', '4', '4', '3', '1', '1', '2', '4', '5', '4', '5', '2', '1', '3', '1', '2', '5', '5', '5', '4', '1', '5', '5', '3', '2', '4', '1', '5', '1', '3', '4', '1', '1', '2', '5', '1', '2', '1', '5', '3', '2', '3', '5', '4', '1', '4', '1', '3', '5', '5', '4', '5', '3', '4', '5', '4', '2', '5', '1', '3', '2', '1', '4', '4', '4', '4', '4', '5', '3', '4', '1', '4', '5', '4', '5', '1', '2', '1', '3', '3', '1', '1', '1', '4', '3', '5', '4', '5', '5', '1', '5', '5', '1', '3', '4', '1', '3', '5', '3', '4', '1', '1', '5', '1', '4', '1', '1', '3', '5', '5', '1', '4', '2', '2', '4', '2', '3', '5', '5', '4', '2', '5', '5', '3', '3', '4', '5', '5', '5', '4'

In [16]:
docs = [l.split() for l in datalines]
print(len(docs))
print(docs[0])

14438
['Catheterization', 'laboratory', 'events', 'and', 'hospital', 'outcome', 'with', 'direct', 'angioplasty', 'for', 'acute', 'myocardial', 'infarction', 'To', 'assess', 'the', 'safety', 'of', 'direct', 'infarct', 'angioplasty', 'without', 'antecedent', 'thrombolytic', 'therapy,', 'catheterization', 'laboratory', 'and', 'hospital', 'events', 'were', 'assessed', 'in', 'consecutively', 'treated', 'patients', 'with', 'infarctions', 'involving', 'the', 'left', 'anterior', 'descending', '(n', '=', '100', 'patients),', 'right', '(n', '=', '100),', 'and', 'circumflex', '(n', '=', '50)', 'coronary', 'arteries.', 'The', 'groups', 'of', 'patients', 'were', 'similar', 'for', 'age', '(left', 'anterior', 'descending', 'coronary', 'artery,', '59', 'years;', 'right', 'coronary', 'artery,', '58', 'years;', 'circumflex', 'coronary', 'artery,', '62', 'years),', 'patients', 'with', 'multivessel', 'disease', '(left', 'anterior', 'descending', 'coronary', 'artery,', '55%;', 'right', 'coronary', 'artery,

In [17]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]
docs1 = filterLen(docs, 4)
print(len(docs1))
print(len(docs[0]), docs[0][:20])
print(len(docs1[0]), docs1[0][:20])

14438
(234, ['Catheterization', 'laboratory', 'events', 'and', 'hospital', 'outcome', 'with', 'direct', 'angioplasty', 'for', 'acute', 'myocardial', 'infarction', 'To', 'assess', 'the', 'safety', 'of', 'direct', 'infarct'])
(175, ['Catheterization', 'laboratory', 'events', 'hospital', 'outcome', 'with', 'direct', 'angioplasty', 'acute', 'myocardial', 'infarction', 'assess', 'safety', 'direct', 'infarct', 'angioplasty', 'without', 'antecedent', 'thrombolytic', 'therapy,'])


In [18]:
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empty=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empty:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [19]:
mat = build_matrix(docs)
mat1 = build_matrix(docs1)
csr_info(mat, non_empty=True)
csr_info(mat1, non_empty=True)

 [nrows 14438 (14438 non-empty), ncols 107112 (107112 non-empty), nnz 1593818]
 [nrows 14438 (14438 non-empty), ncols 101110 (101110 non-empty), nnz 1245864]


In [20]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
mat2 = csr_idf(mat1, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)
print("mat1:", mat1[15,:20].todense(), "\n")
print("mat2:", mat2[15,:20].todense(), "\n")
print("mat3:", mat3[15,:20].todense())

('mat1:', matrix([[ 0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  3.]]), '\n')
('mat2:', matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.41636787,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  1.32309815]]), '\n')
('mat3:', matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.00707066,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.02246854]]))


In [22]:
def splitData(mat, cls, fold=1, d=10):
    r""" Split the matrix and class info into train and test data using d-fold hold-out"""
    n = mat.shape[0]
    r = int(np.ceil(n*1.0/d))
    mattr = []
    clstr = []
    # split mat and cls into d folds
    for f in range(d):
        if f+1 != fold:
            mattr.append( mat[f*r: min((f+1)*r, n)] )
            clstr.extend( cls[f*r: min((f+1)*r, n)] )
    # join all fold matrices that are not the test matrix
    train = sp.vstack(mattr, format="csr")
    # extract the test matrix and class values associated with the test rows
    test = mat[(fold-1)*r: min(fold*r, n), :]
    clste = cls[(fold-1)*r: min(fold*r, n)]

    return train, clstr, test, clste
    
def classifyNames(mat, cls = None, k=3, d=10):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
#     docs = [cmer(n, c) for n in names]
#     mat = build_matrix(docs)
    # since we're using cosine similarity, normalize the vectors
#     csr_l2normalize(mat)
    
    def classify(x, train, clstr):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        
    macc = 0.0
    clspr = []
    for f in range(d):
        # split data into training and testing
        train, clstr, test, clste = splitData(mat, cls, f+1, d)
        # predict the class of each test sample
        clspr.extend([ classify(test[i,:], train, clstr) for i in range(test.shape[0]) ])
        
    return clspr
        # compute the accuracy of the prediction
#         acc = 0.0
#         for i in range(len(clste)):
#             if clste[i] == clspr[i]:
#                 acc += 1
#         acc /= len(clste)
#         macc += acc
        
#     return macc/d


In [23]:
result = classifyNames(mat3, classes, k=(int(len(classes)**0.5)))

In [24]:
print(len(result))
outputfile = open('prediction.dat', 'w')
for item in result:
    print(item)
    outputfile.write("%s\n" % item)

14438
4
5
1
4
4
5
5
1
5
5
4
5
2
5
1
1
1
4
1
3
1
5
1
3
4
4
5
5
3
1
5
2
1
4
5
5
1
4
5
5
1
1
4
4
2
1
5
3
1
5
1
4
5
4
5
5
1
4
3
1
5
5
5
5
3
5
1
3
4
4
4
4
3
1
1
1
4
4
4
1
5
5
5
1
5
5
4
4
4
2
5
5
1
2
4
1
5
1
5
4
1
1
2
1
1
5
2
5
3
5
3
3
1
1
4
1
5
5
1
4
5
4
4
5
4
1
5
1
4
2
1
4
4
4
4
4
3
5
4
1
5
1
4
5
3
2
1
3
2
1
1
1
4
5
5
4
5
1
1
4
2
1
5
4
1
3
5
5
4
1
1
1
1
5
1
5
5
2
4
1
4
1
2
4
1
5
1
5
4
4
4
5
5
3
3
3
2
5
5
4
5
5
5
4
1
1
5
4
4
4
4
4
4
4
1
5
4
1
4
1
5
4
2
1
5
4
1
5
4
3
4
4
4
5
2
3
1
1
1
4
2
4
4
1
5
5
5
4
1
4
5
5
1
4
5
5
1
2
1
4
1
4
1
4
3
2
4
4
4
5
5
1
4
4
5
4
5
4
1
5
5
1
3
5
1
4
5
5
3
4
1
4
1
5
5
4
1
5
5
1
4
5
2
4
5
1
1
1
5
5
5
3
2
5
1
3
4
4
2
5
5
4
5
5
1
1
5
1
4
4
4
4
3
2
1
5
5
2
5
5
5
3
4
5
4
4
4
1
4
5
4
5
1
5
1
3
1
3
4
1
4
3
5
1
5
5
4
3
1
1
4
1
5
4
2
5
4
3
5
1
4
5
4
1
1
4
2
3
2
2
1
1
1
1
5
1
4
4
4
1
4
4
1
5
4
1
5
4
5
3
3
4
1
4
4
1
5
4
1
1
4
1
5
4
4
1
4
5
1
2
5
2
4
4
1
4
4
1
1
4
1
4
5
5
4
4
5
4
2
5
5
5
5
4
2
5
1
2
4
1
5
1
1
1
5
5
1
5
5
4
5
4
4
1
2
4
4
4
4
4
1
5
4
1
4
1
4
4
4
4
4
5
4
4
1
4
2


3
5
4
4
4
4
3
4
3
2
1
5
5
1
5
5
1
1
1
1
4
1
4
5
5
1
4
4
4
4
1
4
5
5
4
1
5
2
4
4
3
1
4
1
1
1
5
1
5
2
5
5
1
1
4
5
4
1
3
4
1
3
4
4
5
5
4
1
4
4
5
5
4
5
5
5
2
5
4
2
3
1
5
1
4
4
1
2
1
1
5
5
1
5
4
5
2
1
5
4
5
1
4
1
4
5
5
5
1
2
5
3
5
4
2
4
4
1
1
2
4
5
5
1
5
5
2
2
1
5
5
4
4
1
5
3
5
4
4
5
5
5
1
4
5
5
5
4
1
5
1
5
4
5
3
4
1
4
1
5
5
4
2
5
1
5
1
4
3
5
5
1
1
2
3
5
5
2
5
5
4
1
4
3
1
5
5
4
1
5
5
4
4
4
2
5
1
5
1
5
1
1
2
5
4
3
5
1
4
5
5
5
5
3
4
1
2
3
4
1
4
5
5
5
4
5
1
5
4
1
2
5
4
1
2
5
4
5
3
5
1
5
1
1
5
1
4
1
5
1
5
1
5
3
1
5
3
3
1
5
1
5
2
1
4
5
3
2
4
1
3
4
5
5
3
4
1
4
1
4
1
5
4
5
5
5
4
5
1
5
5
4
4
1
3
1
1
1
1
2
5
5
5
2
1
1
5
5
2
3
3
3
1
4
4
5
1
1
5
1
1
5
5
5
3
5
5
2
1
4
5
5
5
5
1
5
2
1
5
5
1
5
4
1
5
1
3
4
4
1
2
5
2
1
1
5
1
5
5
3
1
4
5
1
3
4
1
4
5
2
4
3
4
1
1
1
1
5
1
4
1
1
2
4
5
1
4
5
4
2
2
3
5
4
4
5
5
5
1
5
1
1
3
5
5
2
4
3
4
5
1
1
5
4
4
5
5
4
1
4
4
1
4
4
5
3
5
5
1
1
1
4
1
4
4
5
5
5
4
3
5
4
4
3
1
3
4
4
4
5
2
1
5
2
5
4
4
5
1
1
4
5
4
5
5
5
4
1
5
5
5
4
5
4
5
4
4
5
1
5
4
4
5
1
4
2
5
1
2
5
1
1
5
5
2
5
1
3
1
1


5
5
5
4
4
2
1
5
5
5
5
4
5
4
5
2
4
4
4
5
5
4
5
5
5
4
5
1
5
5
1
4
5
4
5
4
5
5
4
5
4
4
1
4
5
4
5
1
5
4
4
5
2
5
2
5
1
3
3
5
5
4
2
5
5
3
1
5
1
3
1
4
5
1
5
4
5
1
4
1
5
4
4
5
4
1
3
5
5
2
5
2
1
5
3
1
1
2
5
2
4
2
5
1
1
4
2
1
1
3
3
1
5
5
4
2
2
3
5
5
3
3
1
1
1
2
5
5
5
5
1
2
4
2
1
5
4
3
1
4
2
3
2
4
4
1
5
3
1
4
5
5
4
2
3
5
5
1
4
5
4
4
5
1
4
3
1
4
5
5
5
4
1
1
5
1
5
4
4
4
5
1
1
1
4
2
1
5
5
5
5
1
1
5
3
1
4
1
3
1
5
5
4
5
4
1
4
1
4
5
5
5
4
5
5
5
1
5
1
5
5
1
4
1
1
5
2
5
5
1
1
4
4
5
1
5
1
4
4
4
5
1
5
5
1
4
5
1
2
1
1
4
5
5
5
4
1
4
3
5
1
1
5
3
5
1
4
5
5
1
4
3
5
2
1
1
3
1
5
1
1
5
1
1
5
4
1
5
1
4
2
5
4
5
4
4
4
5
3
1
4
5
5
4
5
5
1
1
2
3
4
5
2
5
1
1
4
2
5
5
5
5
5
5
5
4
5
5
5
4
4
5
4
4
5
4
1
1
5
4
1
1
5
1
2
5
5
4
4
1
1
4
5
3
1
5
4
5
2
2
5
4
5
4
4
5
4
4
1
2
4
1
4
2
1
2
4
1
5
1
2
5
5
5
3
4
2
1
4
1
1
2
2
3
1
5
1
4
4
5
1
1
2
3
4
5
1
4
1
1
4
1
4
4
5
4
3
5
5
4
5
5
5
5
5
1
5
4
5
1
1
4
5
3
1
1
5
1
4
3
1
4
1
4
1
5
4
4
2
3
1
5
2
5
1
4
5
5
2
5
2
2
4
5
3
2
3
1
5
1
5
4
5
1
5
1
5
1
5
3
4
5
1
4
5
5
5
3
5
3
4
5
1
5
4
4
5
5
5
4


1442
